# Solution for Puzzle x

## Imports

In [36]:
import pandas as pd
import os
from tqdm import tqdm

## Input Data

In [45]:
def read_and_parse_irregular_data(file_path: str) -> pd.DataFrame:
    """
    Reads an irregularly formatted text file line by line, parses the data 
    blocks using string splitting, and returns a consolidated Pandas DataFrame.

    Args:
        file_path: The path to the input file (e.g., 'input/test.csv').

    Returns:
        A pandas DataFrame containing the parsed data.
    """
    if not os.path.exists(file_path):
        raise ValueError(f"Error: File not found at '{file_path}'. Please check the path.")
        
    raw_lines = []
    try:
        # Read the entire file content into a list of strings
        with open(file_path, 'r', encoding='utf-8') as file:
            # Strip whitespace and filter out empty lines
            raw_lines = [line.strip() for line in file if line.strip()]
    except Exception as e:
        raise RuntimeError(f"An error occurred while reading the file: {e}")

    parsed_data = []

    # Parse each line string
    for line in raw_lines:

        # Split the line by spaces to separate the data blocks
        parts = line.split()
        
        # Check if the line contains enough data blocks
        if len(parts) < 3:
            print(f"Skipping line due to insufficient data: {line}")
            continue

        light_diagram = parts[0]
        joltage_requirements = parts[-1]
        buttons = parts[1:-1]
        
        # Prepare the data dictionary for the row
        row_dict = {'light_diagram': light_diagram, 'joltage_requirements': joltage_requirements, 'buttons': buttons}
        
        parsed_data.append(row_dict)

    # 3. Create the final DataFrame from the list of dictionaries
    df = pd.DataFrame(parsed_data)
    
    return df

file_to_process = 'input/day_10.csv'
input = read_and_parse_irregular_data(file_to_process)

print("\n--- Resulting DataFrame ---")
print(input.head(5))


--- Resulting DataFrame ---
  light_diagram    joltage_requirements  \
0      [#...##]  {132,30,23,13,121,115}   
1        [##.#]            {2,18,18,20}   
2      [.##...]     {38,30,29,22,13,25}   
3      [#..###]      {22,51,19,63,22,1}   
4      [#.#.##]     {13,43,33,29,40,41}   

                                             buttons  
0         [(0,1,3,4,5), (0,4,5), (1,2,3,4), (0,1,2)]  
1                                [(0,3), (1), (2,3)]  
2    [(0,1,5), (0,2,3,5), (0,3,4), (1,2), (0,1,2,4)]  
3  [(0,5), (1,3), (1,3,4), (0,1,2,3), (3), (0,1,3...  
4  [(2,3,4,5), (0,2,4,5), (0,1,4), (1,5), (1,2,3,...  


## Part One

In [57]:
def apply_button(status, button):
    """Applies button on all lamps in status."""
    button = button[1:-1] #strip ()
    lamps = button.split(",")
    for lamp in lamps:
        status[int(lamp)] *= -1
    return status

def bfs(curr_status, goal_status, buttons):
    """Performs BFS where buttons are vertices and lamp status lists the nodes."""
    seen = set()
    stack = [(curr_status, 0)]
    while stack:
        curr, curr_count = stack.pop(0)
        if curr == goal_status:
            return curr_count
        for button in buttons:
            new_status = apply_button(curr.copy(), button)
            if tuple(new_status) not in seen:
                seen.add(tuple(new_status))
                stack.append((new_status, curr_count+1))
    return -1

number_buttons_pressed = []
for idx, row in tqdm(input.iterrows(), total=len(input)):
    status_lights = [-1 for _ in range(len(row["light_diagram"])-2)]
    status_lights_goal = [-1 if status == "." else 1 for status in row["light_diagram"][1:-1]]
    buttons = row["buttons"]
    number_buttons_pressed_row = bfs(status_lights, status_lights_goal, buttons)
    number_buttons_pressed.append(number_buttons_pressed_row)
print(f"The fewest button presses required are {sum(number_buttons_pressed)}")
        

100%|██████████| 187/187 [00:00<00:00, 1498.50it/s]

The fewest button presses required are 491


## Part Two